# Import

In [1]:
import gdown

# Specify the file ID and output path
file_id = "1-0A3obqFnABamVwIRGBKTgG2mQ1mrtI2"
destination = "/content/train_data.csv"  # Change the filename and extension if necessary

# Download the file
gdown.download(f"https://drive.google.com/uc?id={file_id}", destination, quiet=False)

print("File downloaded to:", destination)

Downloading...
From: https://drive.google.com/uc?id=1-0A3obqFnABamVwIRGBKTgG2mQ1mrtI2
To: /content/train_data.csv
100%|██████████| 7.75M/7.75M [00:00<00:00, 57.6MB/s]

File downloaded to: /content/train_data.csv


In [2]:

import gdown

# Specify the file ID and output path
file_id = "1do5Bafv-8tIJ72U5trb1MLJrtUqUxoxN"
destination = "/content/test_data.csv"  # Change the filename and extension if necessary

# Download the file
gdown.download(f"https://drive.google.com/uc?id={file_id}", destination, quiet=False)

print("File downloaded to:", destination)

Downloading...
From: https://drive.google.com/uc?id=1do5Bafv-8tIJ72U5trb1MLJrtUqUxoxN
To: /content/test_data.csv
100%|██████████| 3.18M/3.18M [00:00<00:00, 107MB/s]

File downloaded to: /content/test_data.csv


In [3]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import accuracy_score

In [4]:
# # # Define preprocessing functions
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [5]:
train_data = pd.read_csv('/content/train_data.csv')
test_data = pd.read_csv('/content/test_data.csv')
#test_data = None

In [6]:
train_data.head()

,SampleID,Discussion,Category
0,1,"Without sitting down and doing it manually, yo...",Sports
1,2,All your Search ends with this link.,STEM
2,3,"No, the program you're using is made to be com...",STEM
3,4,Mike Woicik\n\nThe correct answer is: Mike Woi...,Sports
4,5,"No, but not because of why you might think. Wh...",Politics


In [7]:
# test_data.head(3)

# Preprocessing

In [8]:
!pip install nltk
import nltk
nltk.download('punkt_tab')
nltk.download('words')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [9]:
# Step 1: Download GloVe embeddings
!wget http://nlp.stanford.edu/data/glove.6B.zip

# Step 2: Unzip the downloaded file
!unzip glove.6B.zip

# Step 3: Verify the files
!ls -lh glove.6B.*



--2024-12-10 22:02:09--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-12-10 22:02:09--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-12-10 22:02:09--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [40]:
import Preprocessing  # Import the entire module
import importlib
importlib.reload(Preprocessing)  # Reload the module
from Preprocessing import preprocess


In [11]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization
from tensorflow.keras.models import Model

In [12]:
from tensorflow.keras.layers import (
    Input, Embedding, Dense, Dropout,
    MultiHeadAttention, LayerNormalization, GlobalAveragePooling1D
)
from tensorflow.keras.models import Model

def create_transformer_model(embedding_matrix, max_sequence_length, embedding_dim, num_heads=4, ff_dim=128, num_classes=5):
    input_layer = Input(shape=(max_sequence_length,))

    # Embedding layer with pre-trained GloVe embeddings
    embedding_layer = Embedding(
        input_dim=embedding_matrix.shape[0],
        output_dim=embedding_dim,
        weights=[embedding_matrix],
        input_length=max_sequence_length,
        trainable=False
    )(input_layer)

    # Transformer block
    attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)(
        query=embedding_layer, key=embedding_layer, value=embedding_layer
    )
    attention_output = LayerNormalization(epsilon=1e-6)(attention_output + embedding_layer)

    ff_output = Dense(ff_dim, activation='relu')(attention_output)
    ff_output = Dense(embedding_dim)(ff_output)
    ff_output = LayerNormalization(epsilon=1e-6)(ff_output + attention_output)

    # Pooling and output
    global_avg_pool = GlobalAveragePooling1D()(ff_output)
    dropout_layer = Dropout(0.3)(global_avg_pool)
    output_layer = Dense(num_classes, activation='softmax')(dropout_layer)

    model = Model(inputs=input_layer, outputs=output_layer)
    return model


In [41]:

glove_path = '/content/glove.6B.100d.txt'
embedding_dim = 100
pre_method = 2
fx_opt = 3
X_train, Y_train, X_test, embedding_matrix, max_sequence_length = preprocess(
    train_data, test_data, pre_method, fx_opt, glove_path, embedding_dim
)


Drop Nan...
	train_data.shape before (24989, 3)
	train_data.shape after (24646, 3)
start preprocessing...
Encoding Y_train...
Unique words in the dataset:
['a', 'aa', 'aardvark', 'aba', 'abacus', 'abandon', 'abandoned', 'abandonment', 'abas', 'abate', 'abatement', 'abb', 'abbey', 'abbot', 'abbreviate', 'abbreviation', 'abdominal', 'aberrant', 'aberration', 'abhor', 'abidal', 'abide', 'abiding', 'abigail', 'ability', 'abject', 'abjure', 'ablaze', 'able', 'abnormal', 'abnormality', 'abnormally', 'aboard', 'abode', 'abolish', 'abolition', 'abomination', 'aborad', 'abord', 'aboriginal', 'abort', 'aborted', 'abortion', 'abound', 'abreast', 'abroad', 'abruptly', 'absence', 'absent', 'absentee', 'absinthe', 'absinthium', 'absolute', 'absolutely', 'absolve', 'absorb', 'absorbed', 'absorbing', 'abstain', 'abstinence', 'abstract', 'abstraction', 'absurd', 'absurdly', 'abu', 'abundance', 'abundant', 'abuse', 'abuser', 'abusive', 'abut', 'aby', 'abyss', 'academe', 'academic', 'academy', 'accelerat

In [42]:
print(max_sequence_length)

397


In [44]:
model_2 = create_transformer_model(
    embedding_matrix, max_sequence_length, embedding_dim, num_classes=5
)

model_2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_2.summary()



Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 397)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_2 (Embedding)   │ (None, 397, 100)       │      1,696,100 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_2    │ (None, 397, 100)       │        161,300 │ embedding_2[0][0],     │
│ (MultiHeadAttention)      │                        │                │ embedding_2[0][0],     │
│                           │                        │                │ embedding_2[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_4 (Add)               │ (None, 397, 100)       │              0 │ multi_head_attention_… │
│                           │                        │                │ embedding_2[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_4     │ (None, 397, 100)       │            200 │ add_4[0][0]            │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_6 (Dense)           │ (None, 397, 128)       │         12,928 │ layer_normalization_4… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_7 (Dense)           │ (None, 397, 100)       │         12,900 │ dense_6[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_5 (Add)               │ (None, 397, 100)       │              0 │ dense_7[0][0],         │
│                           │                        │                │ layer_normalization_4… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_5     │ (None, 397, 100)       │            200 │ add_5[0][0]            │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling1d… │ (None, 100)            │              0 │ layer_normalization_5… │
│ (GlobalAveragePooling1D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_5 (Dropout)       │ (None, 100)            │              0 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_8 (Dense)           │ (None, 5)              │            505 │ dropout_5[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 1,884,133 (7.19 MB)

 Trainable params: 188,033 (734.50 KB)

 Non-trainable params: 1,696,100 (6.47 MB)

In [49]:
model_2.fit(X_train, Y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
617/617 ━━━━━━━━━━━━━━━━━━━━ 14s 23ms/step - accuracy: 0.7209 - loss: 0.7154 - val_accuracy: 0.6416 - val_loss: 0.9938
Epoch 2/10
617/617 ━━━━━━━━━━━━━━━━━━━━ 14s 23ms/step - accuracy: 0.7170 - loss: 0.7116 - val_accuracy: 0.6396 - val_loss: 1.0284
Epoch 3/10
617/617 ━━━━━━━━━━━━━━━━━━━━ 14s 23ms/step - accuracy: 0.7252 - loss: 0.6952 - val_accuracy: 0.6450 - val_loss: 1.0475
Epoch 4/10
617/617 ━━━━━━━━━━━━━━━━━━━━ 20s 23ms/step - accuracy: 0.7224 - loss: 0.7039 - val_accuracy: 0.6377 - val_loss: 1.0513
Epoch 5/10
617/617 ━━━━━━━━━━━━━━━━━━━━ 20s 23ms/step - accuracy: 0.7205 - loss: 0.6919 - val_accuracy: 0.6290 - val_loss: 1.0341
Epoch 6/10
617/617 ━━━━━━━━━━━━━━━━━━━━ 14s 23ms/step - accuracy: 0.7220 - loss: 0.6935 - val_accuracy: 0.6375 - val_loss: 1.0350
Epoch 7/10
617/617 ━━━━━━━━━━━━━━━━━━━━ 14s 23ms/step - accuracy: 0.7239 - loss: 0.6873 - val_accuracy: 0.6325 - val_loss: 1.0274
Epoch 8/10
617/617 ━━━━━━━━━━━━━━━━━━━━ 16s 25ms/step - accuracy: 0.7319 - loss: 0.6725 - 

In [48]:
file_name = f'transformer-golve-len(seq=max)-new_preproccing-{pre_method}-e{10}-a{int(0.7107*100)}'
model_2.save(f'{file_name}.h5')

In [ ]:
print(f"X_train.shape: {X_train_padded.shape}")
# print(f"X_test.shape: {X_test_padded.shape}")
print(f"Y_train.shape: {Y_train.shape}")

X_train.shape: (24646, 809)
Y_train.shape: (24646,)


### Test

In [ ]:
# Y_test_pred = ffnn_model.predict(X = X_test)

In [ ]:
# file_name = f'FFNN-m{pre_method}-e{epochs}'
# save_csv(data=Y_test_pred, file_name=file_name, header=['SampleID', 'Category'], numbering=True)

In [ ]:
from keras.models import load_model

model = load_model("/content/transformer-golve-len(seq=100)-2-e10-a75.h5")


TypeError: too many positional arguments

In [ ]:
T_model = create_transformer_model(
    embedding_matrix, max_sequence_length, embedding_dim, num_classes=5
)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
T_model.load_weights("/content/transformer-golve-len(seq=100)-2-e10-a75.h5")

In [ ]:
T_model.predict(X_train)

771/771 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step


array([[9.01784841e-03, 3.50291520e-01, 1.95376445e-02, 6.73211142e-02,
        5.53831816e-01],
       [2.04822004e-01, 1.21993214e-01, 1.82017386e-01, 2.19726354e-01,
        2.71441042e-01],
       [8.54554179e-04, 1.60412153e-03, 1.32182315e-02, 7.52610341e-03,
        9.76797044e-01],
       ...,
       [2.39283517e-01, 1.87819734e-01, 3.07344139e-01, 2.14301154e-01,
        5.12514301e-02],
       [7.28529394e-01, 1.23595644e-03, 9.70531662e-04, 2.25707307e-01,
        4.35568988e-02],
       [8.44024941e-02, 4.44405079e-02, 7.49796927e-01, 1.01607226e-01,
        1.97529048e-02]], dtype=float32)

In [50]:

# Predict
predictions = model_2.predict(X_train)

# Convert predictions to class labels
predicted_classes = predictions.argmax(axis=1)

# Calculate accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(Y_train, predicted_classes)
print(f"Accuracy: {accuracy * 100:.2f}%")




771/771 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step
Accuracy: 72.15%


In [ ]:

test_predictions = model_2.predict(X_test)

Y_test_pred = np.argmax(test_predictions, axis=1)


330/330 ━━━━━━━━━━━━━━━━━━━━ 10s 30ms/step


In [ ]:
import csv

file_name = f'Transformer-glove-maxlen-m{2}-e{10}'
file_path = f"/content/{file_name}.csv"
with open(file_path, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(['SampleID', 'Category'])

    for index, item in enumerate(Y_test_pred, start=1):  # Start numbering from 1
        writer.writerow([index, item])

print(f"Data saved to {file_path}")

Data saved to /content/Transformer-glove-maxlen-m2-e10.csv


In [1]:
import re

In [18]:
def split_and_clean_tokens(tokens):
    cleaned_tokens = []
    for token in tokens:
        # Check if the token contains both numbers and letters
        match = re.match(r'(\d+)?([a-zA-Z]+)?', token)
        if match:
            num_part, word_part = match.groups()
            if num_part:  # Add "number" if numeric part exists
                cleaned_tokens.append("number")
            if word_part:  # Add the word part if it exists
                cleaned_tokens.append(word_part)
    return cleaned_tokens


In [19]:
print(split_and_clean_tokens(["yousef55", "www", "/sdf", "44hisadf", "zz", "don't" , "السلام"]))

['yousef', 'www', 'number', 'hisadf', 'zz', 'don']
